In [88]:
import random
import math

In [89]:
def super_increasing(num):
    """Generate a super-increasing sequence"""
    sequence = [random.randint(1, 10)]  # Start with a smaller number
    for _ in range(num - 1):
        next_num = sum(sequence) + random.randint(1, 10)
        sequence.append(next_num)
    return sequence


In [90]:
def permutation_list(num):
    """Generate a permutation of numbers 0 to num-1"""
    permutation = list(range(num))
    random.shuffle(permutation)
    return permutation

In [91]:
def public_key(sequence, n, r, permutation):
    """Generate public key"""
    temp = [r * sequence[i] % n for i in range(len(sequence))]
    return [temp[permutation[i]] for i in range(len(permutation))]

In [92]:
def private_key(n, r, sequence, permutation):
    """Generate private key"""
    return (n, r, sequence, permutation)

In [93]:
def text_to_binary(text):
    """Convert text to binary string"""
    return ''.join(format(ord(char), '08b') for char in text)

In [94]:
def encrypt(plain_text, public_key):
    """Encrypt the plain text using public key"""
    binary = text_to_binary(plain_text)

    if len(binary) > len(public_key):
        raise ValueError("Message too long for the current key size")

    # Pad binary with zeros if needed
    binary = binary.ljust(len(public_key), '0')

    # Calculate cipher text
    cipher_text = sum(int(binary[i]) * public_key[i] for i in range(len(binary)))
    return cipher_text

In [95]:
def decrypt(cipher_text, private_key):
    """Decrypt the cipher text using private key"""
    n, r, sequence, permutation = private_key

    # Calculate modular multiplicative inverse of r
    r_inverse = pow(r, -1, n)

    # Transform cipher text
    s = (cipher_text * r_inverse) % n

    # Reverse permutation
    inv_perm = [0] * len(permutation)
    for i in range(len(permutation)):
        inv_perm[permutation[i]] = i

    # Decrypt using super-increasing sequence
    binary = ['0'] * len(sequence)
    temp = s

    for i in range(len(sequence) - 1, -1, -1):
        if temp >= sequence[i]:
            binary[inv_perm[i]] = '1'
            temp -= sequence[i]

    # Convert binary back to text (only take complete bytes)
    binary_str = ''.join(binary)
    complete_bytes = binary_str[:len(binary_str) - (len(binary_str) % 8)]
    return binary_to_text(complete_bytes)

In [98]:
def main():
    # Parameters
    message = "hehe catto, my love"
    bits_needed = len(message) * 8  # 8 bits per character - num

    # Generate keys
    sequence = super_increasing(bits_needed)
    n = sum(sequence) * 2  # Make sure n is larger than sum of sequence
    r = random.randint(2, n-1)
    while math.gcd(r, n) != 1:
        r = random.randint(2, n-1)

    permutation = permutation_list(bits_needed)
    pub_key = public_key(sequence, n, r, permutation)
    priv_key = private_key(n, r, sequence, permutation)

    # Encrypt and decrypt
    cipher_text = encrypt(message, pub_key)
    decrypted = decrypt(cipher_text, priv_key)

    print(f"Original message: {message}")
    print(f"Encrypted: {cipher_text}")
    print(f"Decrypted: {decrypted}")

if __name__ == "__main__":
    main()

Original message: hehe catto, my love
Encrypted: 1340065963689923767810173541170650124202849929044
Decrypted: hehe catto, my love
